In [ ]:
from __future__ import print_function
import sys
import random
random.seed(49999)
import numpy

sys.path.append('../matchzoo/inputs/')
sys.path.append('../matchzoo/utils/')
from preparation import Preparation
from preprocess import Preprocess, NgramUtil

In [ ]:

if __name__ == '__main__':
    prepare = Preparation()
    srcdir = '../matchzoo/'
    dstdir = 'D:/nlp_data/'


    ####################
    #input is quora data
    #infile = srcdir + 'quora_duplicate_questions.tsv'
    #corpus, rels = prepare.run_with_one_corpus_for_quora(infile)


    #####################
    #input is SICK data
    infile = srcdir + 'train_snli.txt'

    order=2
    #order=2， sen1,sen2, 0 or 1
    #order=1.  0 or 1, sen1, sen2
    corpus, rels = prepare.run_with_one_corpus(infile,order)
    print('total corpus : %d ...' % (len(corpus)))
    print('total relations : %d ...' % (len(rels)))
    prepare.save_corpus(dstdir + 'corpus.txt', corpus)
    prepare.save_relation(dstdir + 'relation.txt', rels)

    rel_train, rel_valid, rel_test = prepare.split_train_valid_test(rels, [0.8, 0.1, 0.1])
    prepare.save_relation(dstdir + 'relation_train.txt', rel_train)
    prepare.save_relation(dstdir + 'relation_valid.txt', rel_valid)
    prepare.save_relation(dstdir + 'relation_test.txt', rel_test)
    print('Preparation finished ...')

    #filter output stop words
    preprocessor = Preprocess(word_stem_config={'enable': False})
    #preprocessor = Preprocess(word_stem_config={'enable': False},word_filter_config={'enable':False})


    dids, docs = preprocessor.run(dstdir + 'corpus.txt')
    preprocessor.save_word_dict(dstdir + 'word_dict.txt')
    preprocessor.save_words_stats(dstdir + 'word_stats.txt')
    fout = open(dstdir + 'corpus_preprocessed.txt', 'w')
    for inum, did in enumerate(dids):
        fout.write('%s\t%s\n' % (did, ' '.join(map(str, docs[inum]))))
    fout.close()
    print('preprocess finished ...')
    print('preprocess finished ...')


In [ ]:
docs

In [ ]:
#word embedding
#w2v_file = sys.argv[1]  # w2v_file



In [ ]:
# coding: utf8
from __future__ import print_function

import sys
import six
import io
import array
import numpy as np
from tqdm import tqdm
import pickle
from gensim.models.wrappers import FastText


def load_word_dict(word_map_file):
    """ file -> {word: index} """
    word_dict = {}
    for line in tqdm(io.open(word_map_file, encoding='utf8')):
        line = line.split()
        try:
            word_dict[line[0]] = int(line[1])
        except:
            print(line)
            continue
    return word_dict
def load_word_embedding(vocab, w2v_file, file_format):
    """
    Pros:
        Save the oov words in oov.p for further analysis.
    Refs:
        class Vectors, https://github.com/pytorch/text/blob/master/torchtext/vocab.py
    Args:
        vocab: dict,
        w2v_file: file, path to file of pre-trained word2vec/glove/fasttext
    Returns:
        vectors
    """
    if (file_format=='text'):
        pre_trained = {}
        n_words = len(vocab)
        embeddings = None  # (n_words, n_dim)

        # str call is necessary for Python 2/3 compatibility, since
        # argument must be Python 2 str (Python 3 bytes) or
        # Python 3 str (Python 2 unicode)
        vectors, dim = array.array(str('d')), None

        # Try to read the whole file with utf-8 encoding.
        binary_lines = False
        try:
            with io.open(w2v_file, encoding="utf8") as f:
                lines = [line for line in f]
        # If there are malformed lines, read in binary mode
        # and manually decode each word from utf-8
        except:
            print("Could not read {} as UTF8 file, "
                  "reading file as bytes and skipping "
                  "words with malformed UTF8.".format(w2v_file))
            with open(w2v_file, 'rb') as f:
                lines = [line for line in f]
            binary_lines = True

        print("Loading vectors from {}".format(w2v_file))

        for line in tqdm(lines):
            # Explicitly splitting on " " is important, so we don't
            # get rid of Unicode non-breaking spaces in the vectors.
            entries = line.rstrip().split(b" " if binary_lines else " ")

            word, entries = entries[0], entries[1:]
            if dim is None and len(entries) > 1:
                dim = len(entries)
                # init the embeddings
                embeddings = np.random.uniform(-0.25, 0.25, (n_words, dim))

            elif len(entries) == 1:
                print("Skipping token {} with 1-dimensional "
                      "vector {}; likely a header".format(word, entries))
                continue
            elif dim != len(entries):
                raise RuntimeError(
                    "Vector for token {} has {} dimensions, but previously "
                    "read vectors have {} dimensions. All vectors must have "
                    "the same number of dimensions.".format(word, len(entries), dim))

            if binary_lines:
                try:
                    if isinstance(word, six.binary_type):
                        word = word.decode('utf-8')

                except:
                    print("Skipping non-UTF8 token {}".format(repr(word)))
                    continue

            if word in vocab and word not in pre_trained:
                embeddings[vocab[word]] = [float(x) for x in entries]
                pre_trained[word] = 1

        # init tht OOV word embeddings
        for word in vocab:
            if word not in pre_trained:
                alpha = 0.5 * (2.0 * np.random.random() - 1.0)
                curr_embed = (2.0 * np.random.random_sample([dim]) - 1.0) * alpha
                embeddings[vocab[word]] = curr_embed

        pre_trained_len = len(pre_trained)
        print('Pre-trained: {}/{} {:.2f}'.format(pre_trained_len, n_words, pre_trained_len * 100.0 / n_words))

        oov_word_list = [w for w in vocab if w not in pre_trained]
        print('oov word list example (30): ', oov_word_list[:30])
        pickle.dump(oov_word_list, open('oov.p', 'wb'), protocol=2)

        embeddings = np.array(embeddings, dtype=np.float32)
        return embeddings
    elif (file_format=='bin'):
        dim=w2v_file['the'].shape[0]
        print(dim)
        #pre_trained = {}
        n_words = len(vocab)
        print(n_words)
        embeddings = np.random.uniform(-0.25, 0.25, (n_words, dim))
        for word in vocab:
            t=vocab[word]
            try:
                embeddings[t]=model[word]
            except:
                alpha = 0.5 * (2.0 * np.random.random() - 1.0)
                curr_embed = (2.0 * np.random.random_sample([dim]) - 1.0) * alpha
                embeddings[t] = curr_embed
                
        embeddings = np.array(embeddings, dtype=np.float32)
        return embeddings
        


In [ ]:
#word embedding
#w2v_file = sys.argv[1]  # w2v_file
w2v_file = "D:/simple_vec/wiki.simple.bin"  # w2v_file
#word_dict_file = sys.argv[2]  # word_dict_file
word_dict_file="D:/nlp_data/word_dict.txt"
#mapped_w2v_file = "embed_fasttext_d300" # output shared w2v dict


print("Fasttext model loading....")
model=FastText.load_fasttext_format(w2v_file )
print("Model Loaded")

word_dict = {}
print('load word dict ...')
word_dict = load_word_dict(word_dict_file)
print('load word vectors ...')
file_format='bin'
embeddings = load_word_embedding(word_dict, model,file_format)
mapped_w2v_file = "embed_fasttext_d300" # output shared w2v dict
print('save word vectors ...')
with open(mapped_w2v_file, 'w') as fw:
    # assert word_dict
    for w, idx in tqdm(word_dict.items()):
        print(word_dict[w], ' '.join(map(str, embeddings[idx])), file=fw)

print('Map word vectors finished ...')


In [ ]:
# coding: utf8
from __future__ import print_function

import os
import sys
import codecs
import numpy
import math

infile = mapped_w2v_file

outfile = "embed_fasttext_d300_norm"
fout = codecs.open(outfile,'w', encoding='utf8')
with codecs.open(infile, 'r', encoding='utf8') as f:
    for line in f:
        r = line.split()
        w = r[0]
        try:
            # BUG: it will happen `name@domain.com`
            vec = [float(k) for k in r[1:]]
        except:
            print(line)
        sum = 0.0
        for k in vec:
            sum += k * k
        sum = math.sqrt(sum)
        for i,k in enumerate(vec):
            vec[i] /= sum
        print(w, ' '.join(['%f' % k for k in vec]), file=fout)
fout.close()

In [ ]:
#!/usr/bin/env python
# coding: utf-8
"""
Generate bin sum used in the attention based neural matching model (aNMM)
"""
from __future__ import print_function

import os
import sys
sys.path.append('../../matchzoo/utils/')
sys.path.append('../../matchzoo/inputs/')
from preprocess import cal_binsum
from rank_io import *


if __name__ == '__main__':
    bin_num = 100
    srcdir = './'
    embedfile = srcdir + 'embed_fasttext_d300_norm'
    corpusfile =  'D:/nlp_data/corpus_preprocessed.txt'

    relfiles = ['D:/nlp_data/relation_train.txt',
           'D:/nlp_data/relation_valid.txt',
            'D:/nlp_data/relation_test.txt'
            ]
    binfiles = [
            srcdir + 'relation_train.binsum-%d.txt' % bin_num,
            srcdir + 'relation_valid.binsum-%d.txt' % bin_num,
            srcdir + 'relation_test.binsum-%d.txt' % bin_num
            ]
    embed_dict = read_embedding(filename = embedfile)
    print('read embedding finished ...')
    _PAD_ = len(embed_dict)
    embed_size = len(list(embed_dict.values())[0])
    embed_dict[_PAD_] = np.zeros((embed_size, ), dtype=np.float32)
    embed = np.float32(np.random.uniform(-0.2, 0.2, [_PAD_+1, embed_size]))
    embed = convert_embed_2_numpy(embed_dict, embed = embed)

    corpus, _ = read_data(corpusfile)
    print('read corpus finished....')
    for idx, relfile in enumerate(relfiles):
        binfile = binfiles[idx]
        rel = read_relation(relfile)
        fout = open(binfile, 'w')
        for label, d1, d2 in rel:
            assert d1 in corpus
            assert d2 in corpus
            qnum = len(corpus[d1])
            d1_embed = embed[corpus[d1]]
            d2_embed = embed[corpus[d2]]
            curr_bin_sum = cal_binsum(d1_embed, d2_embed, qnum, bin_num)
            curr_bin_sum = curr_bin_sum.tolist()
            fout.write(' '.join(map(str, curr_bin_sum)))
            fout.write('\n')
        fout.close()
    print('generate bin sum finished ...')


In [ ]:
from filtering import normalizeDocument

normalizeDocument("i love you're perfect")

In [ ]:

normalizeDocument("i love you're perfect. this is ok")

In [ ]:
model.init_sims(replace=True)

In [ ]:
word_dict

In [ ]:
c=float('nan') 

In [ ]:
from filtering import normalizeDocument
text="man32 is better"
normalizeDocument(text)

In [ ]:
from __future__ import print_function
import sys
import random
random.seed(49999)
import numpy

sys.path.append('../matchzoo/inputs/')
sys.path.append('../matchzoo/utils/')
from preparation import Preparation
from preprocess import Preprocess, NgramUtil
dstdir = 'D:/nlp_data/'

preprocessor = Preprocess(word_stem_config={'enable': False})
dids, docs = preprocessor.run(dstdir + 'corpus1.txt')

In [ ]:
the32

In [ ]:
preprocessor.save_word_dict(dstdir + 'word_dict1.txt')

In [ ]:
a

In [ ]:
model['is']+model['better']

In [ ]:
try:
    model['1']

except:
    print("done")

In [ ]:
word_dict

In [1]:
import os
import sys
sys.path.append('../matchzoo/utils/')
sys.path.append('../matchzoo/inputs/')
from preprocess import cal_binsum
from rank_io import *

In [2]:
123

123

In [19]:
a.split('.')[-1]

'txt'

In [9]:
w2v_file="word_dict1.txt"

import re


In [15]:
a.findall(".")

AttributeError: 'str' object has no attribute 'findall'

In [14]:
a[a.findLast('.')+1:]

AttributeError: 'str' object has no attribute 'findLast'

In [11]:
file_format=str(w2v_file)[(str(w2v_file)).find('.')+1:]

In [12]:
file_format

'txt'